<a href="https://colab.research.google.com/github/ikramyoumba1/BigDataAanlysis/blob/main/tp02_token_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [3]:

import json
from google.colab import files

In [4]:
!mkdir -p /root/.kaggle
!cp /content/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [6]:
print("Uploaded files:")
for filename in uploaded.keys():
    print(f"✅ {filename}")

Uploaded files:
✅ kaggle.json


In [7]:
!pip install -q kaggle

In [1]:

print("✅ Kaggle API has been successfully set up!")

✅ Kaggle API has been successfully set up!


In [8]:
DATASET_NAME = "mkechinov/ecommerce-behavior-data-from-multi-category-store"
TARGET_FILE = "2019-Oct.csv"

In [10]:
!ls /content/kaggle.json



/content/kaggle.json


In [11]:
!cat /content/kaggle.json


{"username":"youmbaiikram","key":"72d7a2a9198d818e170012f04f2c69b9"}

In [12]:
import json
token = {"username":"youmbaiikram","key":"72d7a2a9198d818e170012f04f2c69b9"}
with open("/content/kaggle.json", "w") as f:
    json.dump(token, f)

In [13]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [14]:
!kaggle datasets list | head


ref                                                           title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset  Global Earthquake-Tsunami Risk Assessment Dataset       16151  2025-10-01 16:35:53.273000           5130        196  1.0              
jockeroika/life-style-data                                    Life Style Data                                       3995645  2025-10-14 13:50:45.303000           6655        156  0.8235294        
ahmadrazakashif/bmw-worldwide-sales-records-20102024          BMW Worldwide Sales Records (2010–2024)                853348  2025-09-20 14:39:45.280000          10634        235  1.0              
jaderz/hospital

In [15]:
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store -p /content/data --unzip


Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
100% 4.29G/4.29G [01:20<00:00, 239MB/s]
100% 4.29G/4.29G [01:20<00:00, 56.9MB/s]


In [16]:
!ls /content/data


2019-Nov.csv  2019-Oct.csv


In [2]:
import pandas as pd
import dask.dataframe as dd
import time
import os

In [4]:
path = "/content/data/2019-Oct.csv"

**Pandas + chunksize**

In [7]:
start = time.time()
chunk_size = 500000
chunks = []

for chunk in pd.read_csv(path, chunksize=chunk_size):
    chunks.append(chunk["price"].mean())

mean_price_chunks = sum(chunks) / len(chunks)
end = time.time()
time_chunks = end - start

df_sample = pd.read_csv(path, nrows=100000)
storage_pandas = df_sample.memory_usage(deep=True).sum() / (1024**3)

print("✅ [1] Pandas + chunksize")
print(f"Average price: {mean_price_chunks:.2f}")
print(f"Time taken: {time_chunks:.2f} sec\n")
print(f"Memory used: {storage_pandas:.2f} GB\n")

✅ [1] Pandas + chunksize
Average price: 290.33
Time taken: 103.54 sec

Memory used: 0.03 GB



***[2] Dask***

In [9]:
start = time.time()
df_dask = dd.read_csv(path)
mean_price_dask = df_dask["price"].mean().compute()
end = time.time()
time_dask = end - start

storage_dask = df_dask.memory_usage(deep=True).sum().compute() / (1024**3)

print("✅ [2] Dask")
print(f"Average price: {mean_price_dask:.2f}")
print(f"Time taken: {time_dask:.2f} sec\n")
print(f"Memory used : {storage_dask:.2f} GB\n")

✅ [2] Dask
Average price: 290.32
Time taken: 100.90 sec

Memory used : 6.16 GB



**# [3] Compression**

In [5]:
compressed_path = "/content/data/2019-Oct-compressed.csv.gz"

start = time.time()
df = pd.read_csv(path)
df.to_csv(compressed_path, index=False, compression='gzip')
end = time.time()
time_compress = end - start

original_size_gb = os.path.getsize(path) / (1024**3)
compressed_size_gb = os.path.getsize(compressed_path) / (1024**3)

print("✅ [3] Compression (gzip)")
print(f"Original size: {original_size_gb:.2f} GB")
print(f"Compressed size: {compressed_size_gb:.2f} GB")
print(f"Compression time: {time_compress:.2f} sec")

compression_ratio = (1 - compressed_size_gb / original_size_gb) * 100
print(f"Compression ratio: {compression_ratio:.1f}% smaller")

✅ [3] Compression (gzip)
Original size: 5.28 GB
Compressed size: 1.58 GB
Compression time: 866.87 sec
Compression ratio: 70.1% smaller


**Comparison of results**

In [10]:
print("=== 🧩 Comparison Summary ===")
print(f"Pandas + chunksize: {time_chunks:.2f}s | {storage_pandas:.2f} GB RAM")
print(f"Dask: {time_dask:.2f}s | {storage_dask:.2f} GB RAM")
print(f"Compression: {time_compress:.2f}s | {compressed_size_gb:.2f} GB on disk")

=== 🧩 Comparison Summary ===
Pandas + chunksize: 103.54s | 0.03 GB RAM
Dask: 100.90s | 6.16 GB RAM
Compression: 866.87s | 1.58 GB on disk
